In [4]:
import chardet
from transformers import pipeline, BertTokenizer
import os
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from pathlib import Path
import pandas as pd
import numpy as np
from collections import defaultdict

In [5]:
# since roBERTa can take only 512 tokens at a single time, this function splits the chunks into texts
def split_text_into_chunks(text, max_tokens=500):
    tokens = tokenizer.encode(text)
    return [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]


In [6]:
main_folder = Path('/Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2')

dict_labels={'LABEL_2': 'Neutral',
'LABEL_1': 'Negative',
'LABEL_0': 'Positive'}

#create model and run in chunks and then take an average of the all the sentiments for all the chunks 

columns = ['Filename','Neutral','Negative','Positive']
df = pd.DataFrame(columns = columns)
tokenizer = AutoTokenizer.from_pretrained("gtfintechlab/FOMC-RoBERTa", do_lower_case=True, do_basic_tokenize=True)
model = AutoModelForSequenceClassification.from_pretrained("gtfintechlab/FOMC-RoBERTa", num_labels=3)
config = AutoConfig.from_pretrained("gtfintechlab/FOMC-RoBERTa")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, config=config, device=0, framework="pt",top_k=None)



for txt_file in main_folder.rglob('*.txt'):  # rglob searches recursively
    print(f'Found file: {txt_file}')
    try:
        with open(txt_file, 'r') as f:
            content = f.read()
            
        token_chunks = split_text_into_chunks(content)

        all_label_scores = []
        file_score = defaultdict(list)
        
        for chunk in token_chunks:
            chunk_content = tokenizer.decode(chunk) 
            scores = classifier([chunk_content], batch_size=128)[0]
            
            for d in scores:
                file_score[dict_labels[d['label']]].append(d['score'])
                
        for key,val in file_score.items():
            file_score[key]=np.array(val).mean()

        num_chunks = len(token_chunks)
        len_df = len(df)
        
        new_row = {
            'Filename': str(txt_file).split('/')[-1], 
            'Neutral': file_score['Neutral'],
            'Negative': file_score['Negative'],
            'Positive': file_score['Positive']
        }
# create a new df to store all the values of these sentiments. 
        
        df.loc[len_df] = new_row
        len_df += 1  
        
    except Exception as e:
        print(f'Error reading file{e}')
        content = None
        

Token indices sequence length is longer than the specified maximum sequence length for this model (13950 > 512). Running this sequence through the model will result in indexing errors


Found file: /Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/FOMC_Press_Conference/FOMCpresconf20240918.txt
Found file: /Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/FOMC_Press_Conference/FOMCpresconf20240501.txt
Found file: /Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/FOMC_Press_Conference/FOMCpresconf20140618.txt
Found file: /Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/FOMC_Press_Conference/FOMCpresconf20211215.txt
Found file: /Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/FOMC_Press_Conference/FOMCpresconf20221102.txt
Found file: /Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/FOMC_Press_Conference/FOMCpresconf20200916.txt
Found file: /Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/FOMC_Press_Conference/FOMCpresconf20130918.txt
Found file: /Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/FOMC_Press_Conference/FOMCpresconf20210127.txt
Found file: /Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/FOMC_Press_Conference/FOMCpresconf20171213.txt
Found file

In [10]:
df.to_csv('/Users/pranaymatalia/Desktop/FRE-GY 7871/HW 2/Data/BERT_data.csv')